In [ ]:
import glob
import pickle

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib inline

# Calibration

In [ ]:
max_nx = 9
max_ny = 6

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

pickle_path = 'wide_dist_pickle.p'

In [ ]:
def findChessboardCorners(img, max_nx, max_ny):
    # Try our best to find the max number of corners
    for nx in range(max_nx, 2, -1):
        for ny in range(max_ny, 2, -1):
            ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
            if ret == True:
                return ret, corners, nx, ny
    return False

fig = plt.figure(figsize=(10, 40))

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.set_xticks(())
    sub1.set_yticks(())
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')
    
    # Find the chessboard corners
    ret, corners, nx, ny = findChessboardCorners(gray, max_nx, max_ny)
    
    # If found, add object points, image points
    if ret == True:
        # prepare object points
        objp = np.zeros((nx*ny,3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the images with corners
        cornergray = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)

        sub2 = plt.subplot(len(images), 2, (idx*2)+2)
        sub2.set_xticks(())
        sub2.set_yticks(())
        sub2.imshow(cornergray)
        if idx == 0:
            sub2.set_title('Original Images with corners')

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 40))

img = mpimg.imread(images[0])

# Find out the calibration matrix
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.set_xticks(())
    sub1.set_yticks(())
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')

    # Do camera calibration given object points and image points
    dst = cv2.undistort(img, mtx, dist, None, mtx)

    # Draw and display the undistorted images
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.set_xticks(())
    sub2.set_yticks(())
    sub2.imshow(dst)
    if idx == 0:
        sub2.set_title('Undistorted Images')

fig.tight_layout()
plt.show()

In [ ]:
dist_pickle = {}
dist_pickle['mtx'] = mtx
dist_pickle['dist'] = dist
pickle.dump( dist_pickle, open( pickle_path, 'wb' ) )

# Testing

In [ ]:
images = glob.glob('test_images/test*.jpg')

fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.set_xticks(())
    sub1.set_yticks(())
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')

    # Do camera calibration given object points and image points
    dst = cv2.undistort(img, mtx, dist, None, mtx)

    # Draw and display the undistorted images
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.set_xticks(())
    sub2.set_yticks(())
    sub2.imshow(dst)
    if idx == 0:
        sub2.set_title('Undistorted Images')

fig.tight_layout()
plt.show()